# K-최근점 이웃 회귀 (KNN-Regression) 
- 샘플에 가장 가까운 샘플 k개 선택. 회귀이기에 이웃한 샘플의 타깃은 어떤 클래스가 아니라 임의의 수치임. 이 수치들의 평균을 구하여 새로운 샘플의 타깃값 예측.
- 분류에서는 이웃의 레이블 개수를 확인해서 다수결로 정했지만, 회귀에서는 이웃들의 평균을 계산한다는 점에서 차이가 있음.    
- https://rebro.kr/184

In [1]:
from sklearn.neighbors import KNeighborsRegressor

regress = KNeighborsRegressor(n_neighbors=5, weights='uniform', algorithm='auto', 
                              leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=None)

### KNeighborsRegressor Classe의 기본 구성
- n_neighbors: int, 이웃의 수인 K를 결정한다. default=5
- weights: {'uniform', 'distance} or callable. default=uniform   
    uniform: 각각의 이웃이 모두 동일한 가중치를 갖는다.   
    distance: 거리가 가까울수록 더 높은 가중치를 가져 더 큰 영향을 미치게 된다.   
    callable: 사용자가 직접 정의한 함수를 사용할 수도 있다. 거리가 저장된 배열을 입력으로 받고 가중치가 저장된 배열을 반환하는 함수가 되어야 한다.
- algorithm: auto, ball_tree, kd_tree, brute. default=auto   
    auto: 입력된 훈련데이터에 기반하여 가장 적절한 알고리즘을 사용   
- leaf_size: int, default:30   
    ball-tree나 KD-Tree의 leaf size를 결정한다.   
    이는 트리를 저장하기 위한 메모리뿐만 아니라, 트리의 구성과 쿼리의 처리속도에 영향을 미친다.
- p: int   
    p=1이면, 맨핸튼 거리   
    p=2이면, 유클리드 거리

In [2]:
# 맨해튼 거리
def manhattan_distance(A, B):
    distance = 0
    for i in range(len(A)):
        distance += abs(A[i] - B[i])
    return distance
 
print(manhattan_distance(A=[1, 5, 7, 9], B=[2, 3, 6, 15]))


def euclidean_distance(A, B):
  distance = 0
  for i in range(len(A)):
    distance += (A[i] - B[i]) ** 2
  return distance ** 0.5
  
print(euclidean_distance(A=[1, 5, 7, 9], B=[2, 3, 6, 15]))


10
6.48074069840786


In [2]:
import numpy as np

perch_length = np.array([8.4, 13.7, 15.0, 16.2, 17.4, 18.0, 18.7, 19.0, 19.6, 20.0, 21.0,
       21.0, 21.0, 21.3, 22.0, 22.0, 22.0, 22.0, 22.0, 22.5, 22.5, 22.7,
       23.0, 23.5, 24.0, 24.0, 24.6, 25.0, 25.6, 26.5, 27.3, 27.5, 27.5,
       27.5, 28.0, 28.7, 30.0, 32.8, 34.5, 35.0, 36.5, 36.0, 37.0, 37.0,
       39.0, 39.0, 39.0, 40.0, 40.0, 40.0, 40.0, 42.0, 43.0, 43.0, 43.5,
       44.0])
perch_weight = np.array([5.9, 32.0, 40.0, 51.5, 70.0, 100.0, 78.0, 80.0, 85.0, 85.0, 110.0,
       115.0, 125.0, 130.0, 120.0, 120.0, 130.0, 135.0, 110.0, 130.0,
       150.0, 145.0, 150.0, 170.0, 225.0, 145.0, 188.0, 180.0, 197.0,
       218.0, 300.0, 260.0, 265.0, 250.0, 250.0, 300.0, 320.0, 514.0,
       556.0, 840.0, 685.0, 700.0, 700.0, 690.0, 900.0, 650.0, 820.0,
       850.0, 900.0, 1015.0, 820.0, 1100.0, 1000.0, 1100.0, 1000.0,
       1000.0])

In [3]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(perch_length, perch_weight, random_state = 42)

In [4]:
train_input = train_input.reshape(-1, 1)
test_input = test_input.reshape(-1, 1)

# 결정계수 R^2
- R^2 = 1 - {(타깃-예측)^2/(타깃-평균)^2}

In [5]:
from sklearn.neighbors import KNeighborsRegressor

knr = KNeighborsRegressor()
knr.fit(train_input, train_target)

# 테스트셋의 결정계수
print(knr.score(test_input, test_target)) # 0.992809406101064

0.992809406101064


In [7]:
# 트레이닝셋의 결정 계수
print(knr.score(train_input, train_target))

0.9698823289099254


# 과대적합 vs 과소적합          
- 과소적합: 트레이닝셋보다 테스트셋의 점수가 높거나 두 점수가 모두 너무 낮은경우   
- 과대적합: 트레이닝셋보다 테스트셋의 점수가 지나치게 낮은 경우         

# 해결방법          
- 과소적합의 경우 모델을 조금 더 복잡하게 만든다. 트레이닝셋에 더 잘 맞게 만들면 테스트셋의 점수가 조금 낮아 질 것이다.
- k 값을 조정하여 모델의 복잡도를 바꿀 수 있다. 이웃의 수를 줄이면 트레이닝셋에 있는 국지적인 패턴에 민감해지고, 이웃의 수를 늘리면 데이터 전반에 있는 일반적인 패턴을 따르게 된다.

In [11]:
knr.n_neighbors = 3
knr.fit(train_input, train_target)
print("테스트셋:", knr.score(test_input, test_target)) 
print("트레이닝셋:", knr.score(train_input, train_target))

테스트셋: 0.9746459963987609
트레이닝셋: 0.9804899950518966


In [14]:
# 훈련됭 모델을 이용하여 길이 100인 농어의 무게 예측
print(knr.predict([[100]]))

[1033.33333333]
